In [5]:
import os

BASE_URL = f"http://localhost:8000/v1"

os.environ["BASE_URL"] = BASE_URL
os.environ["OPENAI_API_KEY"] = "abc-123"   

print("Config set:", BASE_URL)

Config set: http://localhost:8000/v1


In [6]:
!pip install -q pydantic_ai openai
!pip install pytz

In [7]:
import os
import json
import asyncio
import pytz
from datetime import datetime, timezone, timedelta
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from typing import Annotated, List
from pydantic_ai import Agent, Tool
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic import BaseModel, Field
from dateutil import parser

In [8]:
@Tool
def get_current_date() -> str:
    """Return the current date/time as an ISO-formatted string."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [9]:
provider = OpenAIProvider(
    base_url=os.environ["BASE_URL"],
    api_key=os.environ["OPENAI_API_KEY"],
)

class MeetingMetadata(BaseModel):
    participants: List[str] = Field(..., description="list of emails who are the participants for the meeting.")
    start_time: str = Field(..., description="A valid format of time string for the Starting point of time window")
    end_time: str = Field(..., description="A valid format of time string for the ending point of time window")
    meeting_duration: int = Field(..., description="Duration of the meeting in minutes.")

@Tool
def get_time_window(
    current_time: Annotated[str, "The string formatted timestamp of current date & time"],
    time_constraints: Annotated[str, "The exact date, day or time window where the call can be scheduled"],
    meeting_duration: Annotated[int, "how long the call/meeting is going to occur, unit in minutes"]
) -> MeetingMetadata:
    """Based on the current_time, time_contstraints, and meeting_duration determine the date range for the required meeting timeframe (start and end date),
        where a work week is defined as Monday to Friday only (excluding weekends).
            Remember the followings -
            - "Next week" always refers to the next Monday–Friday period, not 7 days from current_time.
            - For example, if today is Wednesday, then next week starts from the coming Monday.

        Calculate the start and end timestamps based on the current date, time_constraint and meeting duration given, keeping in mind that it is suppose to be only in work weekdays.
        Get the current day from the input current_date and then understand the actual start date and end date.
        Return all the following:
            The current timestamp
            A valid format of time string (e.g. '2025-07-14 09:00:00', '2025-07-14T03:30:00Z') for the Starting point of timewindow
            A valid format of time string (e.g. '2025-07-14 09:00:00', '2025-07-14T03:30:00Z') for the ending point of time window
            The time_duration for the meeting as it was in the input
    """

In [10]:
class MeetingInfo(BaseModel):
    scheduler: str = Field(..., description="the user email id who is asking for the meeting to be scheduled" )
    participants: str = Field(..., description="Comma-separated emails of all participants.")
    time_constraints: str = Field(..., description="Mentioned timing or date phrase in the email.")
    meeting_duration: int = Field(..., description="Duration of the meeting in minutes.")

@Tool
def extract_meeting_info(
    email: Annotated[str, "The raw email body containing meeting details."]
) -> MeetingInfo:
    """
        Yor are an Agent that helps in scheduling meetings.
        Your job is to extracts Email ID's and Meeting Duration.
        You should return :
        1. List of email id's of participants (comma-separated).
        2. Who is scheduling the call.
        3. Meeting duration in minutes.
        4. Time constraints (e.g., 'next week').
        If the List of email id's of participants are just names, then append @amd.com at the end and return. 
        Return as json with 'scheduler', 'participants', 'time_constraints' & 'meeting_duration'.
        Stricty follow the instructions. Strictly return dict with participents email id's, time constraints & meeting duration in minutes only.
    """

In [11]:
@Tool
def retrive_calendar_events(user: Annotated[str, "The mail id of each individual user."], 
                            start: Annotated[str, "A valid format of time string (e.g., '2025-07-17T00:00:00+05:30') for the Starting point of timewindow"],
                            end: Annotated[str, "A valid format of time string (e.g., '2025-07-17T00:00:00+05:30') for the ending point of time window"]
                           ) -> List[{"StartTime": str, "EndTime": str, "NumAttendees" : int, "Attendees" : List[str], "Summary" : str}]:
    """
        Fetch Google Calendar events for a user between start and end times, the times must be in a valid iso datetime format like (e.g., '2025-07-17T00:00:00+05:30').
        Returns a list of dicts with StartTime, EndTime, NumAttendees, Attendees, Summary.
    """
    events_list = []
    token_path = "../Keys/"+user.split("@")[0]+".token"
    user_creds = Credentials.from_authorized_user_file(token_path)
    calendar_service = build("calendar", "v3", credentials=user_creds)
    events_result = calendar_service.events().list(calendarId='primary', timeMin=start, timeMax=end, singleEvents=True,orderBy='startTime').execute()
    events = events_result.get('items')
    
    for event in events : 
        attendee_list = []
        try:
            for attendee in event["attendees"]: 
                attendee_list.append(attendee['email'])
        except: 
            attendee_list.append("SELF")
        start_time = event["start"]["dateTime"]
        end_time = event["end"]["dateTime"]
        events_list.append(
            {"StartTime" : start_time, 
             "EndTime": end_time, 
             "NumAttendees" :len(set(attendee_list)), 
             "Attendees" : list(set(attendee_list)),
             "Summary" : event["summary"]})
    return events_list

In [12]:
@Tool
def structured_output(event_list: Annotated[List[dict], "A list of dictionaries with detailed calendar status having the following keys - StartTime, EndTime, NumAttendees, Attendees, Summary."],
                     email_content: Annotated[str, "The email content that was given at first"],
                     meeting_start: Annotated[str, "A valid format of time string (e.g., '2025-07-17T00:00:00+05:30') for the start of meeting"],
                     meeting_end: Annotated[str, "A valid format of time string (e.g., '2025-07-17T00:00:00+05:30') for the end of meeting"],
                     duration: Annotated[int, "The duration that was given"]):
    """
        Your job is to structure the output in a json having the following format -
        {
            "Attendees": <a list of dictionaries that has the extracted detialed event_list for each candidate from google calendar>
                example list data : [
                    {
                        "email": <email id of the attendee>
                        "events": [
                            {
                                "StartTime": <ISO Formatted time string like - 2025-07-17T00:00:00+05:30, indicating start time of the meeting>,
                                "EndTime": <ISO Formatted time string like - 2025-07-17T00:00:00+05:30, indicating end time of the meeting>,
                                "NumAttendees": <total no of people attending the meeting>,
                                "Attendees": <A lsit of all attendees, only the email ids>,
                                "Summary": <Summary of the meeting>
                            }
                        ]
                    }...
                ],
            "Subject": <Subject for the meeting that we are processing, possibly a summary of meeting content>,
            "EventStart": <ISO Formatted time string like - 2025-07-17T00:00:00+05:30, indicating start time of the current meeting we are trying to schedule>,
            "EventEnd": <ISO Formatted time string like - 2025-07-17T00:00:00+05:30, indicating end time of the current meeting we are trying to schedule>,
            "Duration_mins": <Total duration of the meeting>,
            "Metadata": <The thought process we have used to find out a suitable time>
        }
    """

In [13]:
@Tool
def parse_to_iso_datetime(date_str: str, timezone_str: str = 'Asia/Kolkata') -> str:
    """
    Converts any human-readable date-time string to ISO format with timezone offset.
    
    Args:
        date_str (str): Input date-time string (e.g., 'next Thursday', 'July 17, 2025').
        timezone_str (str): Timezone name (default: 'Asia/Kolkata').
        
    Returns:
        str: ISO formatted date-time string with offset (e.g., '2025-07-17T00:00:00+05:30').
    """
    # Parse to naive datetime
    dt_naive = parser.parse(date_str, fuzzy=True)

    # Attach timezone
    tz = pytz.timezone(timezone_str)
    dt_localized = tz.localize(dt_naive)

    # Return ISO string with offset
    return dt_localized.isoformat()

In [14]:
agent_model = OpenAIModel("Qwen3-8b", provider=provider)

agent = Agent(
    model=agent_model,
    tools=[extract_meeting_info, get_current_date, get_time_window, parse_to_iso_datetime, retrive_calendar_events, structured_output],
    system_prompt = """
    You are an AI agent that assists with meeting scheduling.
        Your tools include:
            - extract_meeting_info(email)
            - get_current_time()
            - get_time_window()
            - parse_to_iso_datetime()
            - retrive_calendar_events()
            - structured_output()

        Your task is to:
            1. Parse the email
            2. Determine the current date
            3. Find out the required meeting timeframe using get_time_window
            4. Convert the requried timeframe to a proper time format
            5. Retrive the calendar events for the attendees and suggest a correct time for the meeting
            6. If meeting is not possible due to no available time during work hours, find out if any call can be of less prioruty by comparing the current meeting context and the summary of existing meeting, if nothing can be done, please suggest a time in earliest possible date
            7. Finally give a structured json output
    """
)

In [15]:
from flask import Flask, request, jsonify
from threading import Thread
import json

In [16]:
app = Flask(__name__)
received_data = []

In [17]:
async def your_meeting_assistant(data): 
    # Your Agentic AI Calls 
    
    email_text = ""
    email_text += data["EmailContent"]
    email_text += "\\n" + "From:"  
    email_text += data["From"]
    email_text += "\\n" + "Attendees:"
    email_text += ",".join(item["email"] for item in data["Attendees"])
    email_text += f", {data["From"]}"

    print(email_text)
    
    response = await agent.run(f"Suggest a time for meeting as per the following email based on both the attendees's availability, the email given:\n\n{email_text}")
    print(response.output)
    response_json = json.loads(response.output)
    response_json["Request_id"] = data["Request_id"]
    response_json["Datetime"] = data["Datetime"]
    response_json["Location"] = data["Location"]
    response_json["From"] = data["From"]
    response_json["EmailContent"] = data["EmailContent"]

    temp_event_dict = {
        "StartTime": response_json["EventStart"],
        "EndTime": response_json["EventEnd"],
        "NumAttendees": len(data["Attendees"])+1,
        "Attendees": [item["email"] for item in data["Attendees"]].append(data["From"]),
        "Summary": response_json["Subject"]
    }
    
    for attendee in response_json["Attendees"]:
        if attendee['email'] == data["From"]:
            attendee['events'].append(temp_event_dict)
            
    return response_json

In [18]:
@app.route('/receive', methods=['POST'])
def receive():
    data = request.get_json()
    print(f"\n Received: {json.dumps(data, indent=2)}")
    new_data = asyncio.run(your_meeting_assistant(data))  # Your AI Meeting Assistant Function Call
    received_data.append(data)
    print(f"\n\n\n Sending:\n {json.dumps(new_data, indent=2)}")
    return jsonify(new_data)

def run_flask():
    app.run(host='0.0.0.0', port=5000)

In [19]:
# Start Flask in a background thread
Thread(target=run_flask, daemon=True).start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://129.212.176.253:5000
Press CTRL+C to quit



 Received: {
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
  "Datetime": "09-07-2025T12:34:55",
  "Location": "IIT Mumbai",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com"
    },
    {
      "email": "userthree.amd@gmail.com"
    }
  ],
  "Subject": "Agentic AI Project Status Update",
  "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}
Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project.\nFrom:userone.amd@gmail.com\nAttendees:usertwo.amd@gmail.com,userthree.amd@gmail.com, userone.amd@gmail.com


42.104.100.108 - - [13/Jul/2025 07:36:56] "POST /receive HTTP/1.1" 200 -




{
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-16T18:00:00+05:30",
          "EndTime": "2025-07-17T09:00:00+05:30",
          "NumAttendees": 1,
          "Attendees": ["SELF"],
          "Summary": "Off Hours"
        },
        {
          "StartTime": "2025-07-17T18:00:00+05:30",
          "EndTime": "2025-07-18T09:00:00+05:30",
          "NumAttendees": 1,
          "Attendees": ["SELF"],
          "Summary": "Off Hours"
        }
      ]
    },
    {
      "email": "userthree.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-16T16:00:00+05:30",
          "EndTime": "2025-07-17T07:30:00+05:30",
          "NumAttendees": 1,
          "Attendees": ["SELF"],
          "Summary": "Off Hours"
        },
        {
          "StartTime": "2025-07-17T16:00:00+05:30",
          "EndTime": "2025-07-18T07:30:00+05:30",
          "NumAttendees": 1,
          "Attendees": ["SELF"],
       